In [3]:
import torch
from datasets import load_dataset, load_from_disk
import transformers
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
#from peft import LoraConfig, TaskType, get_peft_model
#from peft import PeftConfig, PeftModel
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate

import pandas as pd
import numpy as np
import os

2024-04-26 17:33:21.253601: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 17:33:21.253694: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 17:33:21.386484: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Questions:
# Difference between falcon base and falcon instructed

In [4]:
print(transformers.__version__)

4.39.3


In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [6]:
llm_repo_dir = 'projects/LLM'
cache_dir = 'projects/LLM/.cache'
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_HOME'] = cache_dir + '/huggingface'
os.environ['XDG_CACHE_HOME'] = cache_dir
os.environ['HF_DATASETS_CACHE'] = cache_dir

## mistral
#model_name = "mistralai/Mistral-7B-v0.1"
#model_path = cache_dir + ''

## falcon
model_name = "tiiuae/falcon-7b-instruct"
model_path = cache_dir + '/models--tiiuae--falcon-7b-instruct/snapshots/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99'

## gpt2
#model_name = 'gpt2'
#model_path = cache_dir + '/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e'

## try gpt-neo


dataset_path = cache_dir + '/parquet/yelp_polarity' # cache_dir + '/parquet/yelp_review_full-e22176106d6e7534'
dataset_name = 'yelp_polarity' # yelp_review_full
#tokenized_data_path = cache_dir + '/tokenized_dataset_yelp_polarity_gpt2'
#tokenized_data_path = cache_dir + '/tokenized_dataset_yelp_polarity_mistral7b'
tokenized_data_path = cache_dir + '/tokenized_dataset_yelp_polarity_falcon7b'

In [7]:
if not os.path.isfile(model_path + '/tokenizer.json'):
    tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                              cache_dir=cache_dir,
                                              #torch_dtype=torch.bfloat16,
                                              device_map='auto') # to load tokenizer to cache
else:
    tokenizer = AutoTokenizer.from_pretrained(model_path, 
                                              cache_dir=cache_dir, 
                                              local_flies_only=True)
if not os.path.isfile(model_path + '/model.safetensors'):
    model = AutoModelForCausalLM.from_pretrained(model_name,
                                                 cache_dir=cache_dir,
                                                 device_map='auto')
else:
    model = AutoModelForCausalLM.from_pretrained(model_path, 
                                             cache_dir=cache_dir,
                                             local_files_only=True)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [96]:
prompt = """Comment: “I want to use aooe.account_application_mastertable to map application_id to account_id on AZ 302, but the account_ids were masked. DAAS team provided me with an alternative table which is insurance_ecu.account_application_mastertable_nocc as an alternative. The table only exists on AZ 301 so I have to file a PIA for access to AZ 302. It took a month for DAAS team to go through my access to PIA and another 1 month to go through DAC. Now when I see the data, application_id are all wrong”
What is the sentiment of the comment? Positive, neutral, or negative?"""

# Different tempratures for sentiment analysis

In [111]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
generated_ids = model.generate(input_ids, 
                               do_sample=True, 
                               temperature=0.0001,
                               max_length = 1000)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Comment: “I want to use aooe.account_application_mastertable to map application_id to account_id on AZ 302, but the account_ids were masked. DAAS team provided me with an alternative table which is insurance_ecu.account_application_mastertable_nocc as an alternative. The table only exists on AZ 301 so I have to file a PIA for access to AZ 302. It took a month for DAAS team to go through my access to PIA and another 1 month to go through DAC. Now when I see the data, application_id are all wrong”
What is the sentiment of the comment? Positive, neutral, or negative?
The sentiment of the comment is negative.<|endoftext|>


In [122]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
generated_ids = model.generate(input_ids, 
                               do_sample=True, 
                               temperature=0.6,
                               max_length = 1000)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Comment: “I want to use aooe.account_application_mastertable to map application_id to account_id on AZ 302, but the account_ids were masked. DAAS team provided me with an alternative table which is insurance_ecu.account_application_mastertable_nocc as an alternative. The table only exists on AZ 301 so I have to file a PIA for access to AZ 302. It took a month for DAAS team to go through my access to PIA and another 1 month to go through DAC. Now when I see the data, application_id are all wrong”
What is the sentiment of the comment? Positive, neutral, or negative?
The sentiment of the comment is negative. The commenter is frustrated with the length of time it took to access the data and the fact that they had to file a PIA to access it.<|endoftext|>


In [121]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
generated_ids = model.generate(input_ids, 
                               do_sample=True, 
                               temperature=1,
                               max_length = 1000)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Comment: “I want to use aooe.account_application_mastertable to map application_id to account_id on AZ 302, but the account_ids were masked. DAAS team provided me with an alternative table which is insurance_ecu.account_application_mastertable_nocc as an alternative. The table only exists on AZ 301 so I have to file a PIA for access to AZ 302. It took a month for DAAS team to go through my access to PIA and another 1 month to go through DAC. Now when I see the data, application_id are all wrong”
What is the sentiment of the comment? Positive, neutral, or negative?
The sentiment of the comment is largely positive, with the commentor expressing appreciation for the assistance they received in gaining access to the necessary PIA and DAC processes.<|endoftext|>


In [119]:
input_ids

tensor([[12093,    37,   204,   488,    52,   729,   271,   745,   241,   443,
            80,    25, 18198,    74, 20646,    74, 13166,  7941,   271,  4769,
          2859,    74,   300,   271,  1709,    74,   300,   313, 20389,   204,
         24530,    23,   480,   248,  1709,    74,  1703,   646, 45794,    25,
         14276,  2289,  1255,  2930,   454,   335,   267,  5647,  2409,   585,
           304,  3386,    74, 40301,    25, 18198,    74, 20646,    74, 13166,
          7941,    74,    89, 24430,   345,   267,  5647,    25,   390,  2409,
           736,  8810,   313, 20389,   204, 20879,   485,   295,   413,   271,
          2608,   241,   337,  5172,   312,  1565,   271, 20389,   204, 24530,
            25,   605,  2065,   241,  1325,   312, 14276,  2289,  1255,   271,
           471,   788,   491,  1565,   271,   337,  5172,   273,  1382,   204,
            28,  1325,   271,   471,   788, 54581,    25,  2784,   635,   295,
           760,   248,  1211,    23,  2859,    74,  

In [93]:
tokenizer.decode([12093,    37,   204,   488,    52,   729,   271,   745,   241,   443,
            80,    25, 18198,    74])

'Comment: “I want to use aooe.account_'